- I didn't know much about the field of horse racing, so I explored the data by searching for information on the Internet.
- It's hard to say that there's a clear conclusion, but I hope you enjoy it.
- Distance and speed units were used in kilometers.
- There may be awkward parts in English because I used a translator after organizing them in Korean.

- 경마 분야는 잘 알지 못해 이것저것 정보를 인터넷에서 찾아보며 데이터를 탐험했습니다.
- 뚜렷한 결론이 있다고 말하긴 힘든 노트북이지만, 재밌게 보셨으면 합니다.
- 거리, 속도 단위는 km 단위를 사용했습니다.
- 한글로 정리해둔 뒤 번역기를 썼기 때문에 영문법상 어색한 부분이 있을 수 있습니다.

In [ ]:
# !pip install --upgrade seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import seaborn.objects as so

import scipy.stats as stats

from haversine import haversine

In [ ]:
def preprocess_race():

    race_df = pd.read_csv('../input/big-data-derby-2022/nyra_race_table.csv')

    race_df['race_id'] = race_df[['track_id', 'race_date', 'race_number']].astype('str').agg('_'.join, axis = 1)
    race_df.insert(0, 'race_id', race_df.pop('race_id'))
    
    
    race_df['distance_feet'] = race_df['distance_id'].apply(lambda x : int(6.6 * x)) # to_feet
    race_df['distance_km'] = race_df['distance_feet'].apply(lambda x : x / 3281)

    race_df['race_date'] = pd.to_datetime(race_df['race_date'])
    
    race_df['track_condition'] = race_df['track_condition'].str.replace(" ", "")

    # Dirt Condition Separate 
    race_df.loc[race_df['course_type'] == 'D', 'dirt_condition'] = race_df['track_condition']
    dirt_condition_order = pd.CategoricalDtype(['FT', 'GD', 'MY', 'SY'], ordered = True)
    race_df['dirt_condition'] = race_df['dirt_condition'].astype(dirt_condition_order)
    
    # Turf Condition Separate
    race_df.loc[race_df['course_type'].isin(['I', 'O', 'T']), 'turf_condition'] = race_df['track_condition']
    turf_condition_order = pd.CategoricalDtype(['FM', 'GD', 'YL', 'SF'], ordered = True)
    race_df['turf_condition'] = race_df['turf_condition'].astype(turf_condition_order)
    
    race_df.drop(['purse', 'post_time', 'track_condition'], axis = 1, inplace = True)
    
    # drop hurdle race
    race_df.drop(race_df[race_df['course_type'] == 'M'].index, inplace = True)
    
    return race_df

def preprocess_start():
    start_df = pd.read_csv('../input/big-data-derby-2022/nyra_start_table.csv', 
                        names = ['track_id', 'race_date', 'race_number', 'program_number', 'weight_carried', 'jockey', 'odds', 'position_at_finish'])
    
    start_df['race_date'] = pd.to_datetime(start_df['race_date'])
    start_df['race_id'] = start_df[['track_id', 'race_date', 'race_number']].astype('str').agg('_'.join, axis = 1)

    start_df['program_number'] = start_df['program_number'].str.replace(" ", "")
    return start_df

def preprocess_track():
    track_df = pd.read_csv('../input/big-data-derby-2022/nyra_tracking_table.csv')
    
    track_df['race_date'] = pd.to_datetime(track_df['race_date'])
#     track_df['distance_feet'] = track_df['distance_id'].apply(lambda x : int(6.6 * x))
    track_df['race_id'] = track_df[['track_id', 'race_date', 'race_number']].astype('str').agg('_'.join, axis = 1)

    track_df['program_number'] = track_df['program_number'].str.replace(" ", "")

    return track_df

def preprocess_2022_derby():
    race_df = preprocess_race()
    start_df = preprocess_start()
    track_df = preprocess_track()
    return race_df, start_df, track_df

In [ ]:
race_df, start_df, track_df = preprocess_2022_derby()

# 1. Seasons and Tracks

### Summary
1. Each season consists of Spring (3, 4, 5), Summer (6, 7, 8), Autumn (9, 10, 11), and Winter (12, 1, 2)
2. `Dirt` competition is more than `Turf` competition:
    1. Total number of matches - $Dirt : Turf = 2 : 1$
    2. In terms of seasons, 'Dirt' always has around 300 games, while 'Turf' has a significant decrease in the number of games in spring and winter
    3. About `Dirt`: The condition of the ground often deteriorates in spring and winter, while it is good to run in summer and autumn

### 요약
1. 각 계절은 Spring(3, 4, 5), Summer(6, 7, 8), Autumn(9, 10, 11), Winter(12, 1, 2)로 구성
2. `Dirt` 경기가 `Turf` 경기보다 많음 :
    1. 전체 경기수 - $Dirt : Turf = 2 : 1$
    2. 계절로 놓고 봤을 때, `Dirt`는 항상 300경기 내외의 경기 수가 있는 반면 `Turf`는 봄, 겨울에 경기 수가 확연히 감소함
    3. `Dirt`에 관해 : 봄, 겨울에는 땅의 상태가 달리기 나빠지는 경우가 많고 반면 여름, 가을에는 달리기 좋은 상태
    

- 트랙 컨디션 [위키](https://en.wikipedia.org/wiki/Going_(horse_racing))
    - `Dirt` 
        - `FT : Fast` : 건조하고 균일하며 탄력 있는 표면
        - `GD : Good` : `FT`보다는 느린 듯
        - `MY : Muddy` : 젖었음, 고인 물은 없음
        - `SY : Sloppy` : 고인 물이 보일 정도로 물로 포화된 트랙
    - `Turf`
        - `FM : Firm` : 단단하고 탄력 있는 표면
        - `GD : Good` : `FM`보다 약간 부드러운 잔디 코스
        - `YL : Yielding` : 최근 비가 와서 지면에 상당한 양의 `give(탄성)`가 있음
        - `SF : Soft` : 수분이 많음. 말이 깊게 가라앉을 수 있음

In [ ]:
# 1. Seasons and track conditions
race_df['month'] = race_df['race_date'].dt.month

race_df.loc[race_df['month'].isin([3, 4, 5]), 'season'] = 'spring'
race_df.loc[race_df['month'].isin([6, 7, 8]), 'season'] = 'summer'
race_df.loc[race_df['month'].isin([9, 10, 11]), 'season'] = 'autumn'
race_df.loc[race_df['month'].isin([12, 1, 2]), 'season'] = 'winter'

season_order = pd.CategoricalDtype(['spring', 'summer', 'autumn', 'winter'],
                                        ordered = True)
race_df['season'] = race_df['season'].astype(season_order)

race_df

In [ ]:
dirt_color = ['#783f04', '#bb8632', '#cca466', '#ddc299']
dirt_palette = sns.color_palette(dirt_color, as_cmap = True)

turf_color = ['#38761d', '#739f60', '#afc8a4', '#ebf1e8']
turf_palette = sns.color_palette(turf_color, as_cmap = True)

dirt_turf_color = ['#783f04', '#38761d']
dt_palette = sns.color_palette(dirt_turf_color, as_cmap = True)


In [ ]:
temp_df = race_df.groupby('course_type')['race_id'].count().to_frame().reset_index().rename(columns = {'race_id' : 'races'})
temp_df.loc[temp_df['course_type'] == 'T', 'races'] = temp_df.loc[1:, 'races'].sum()
temp_df.drop(temp_df[temp_df['course_type'].isin(['I', 'O'])].index, inplace = True)

In [ ]:
ax = sns.barplot(data = temp_df, x = 'course_type', y = 'races', palette = ['#783f04', '#38761d'])
ax.bar_label(ax.containers[0])
plt.title('Total Races and Course Type')
plt.figure(figsize = (10, 5))

In [ ]:
temp_df = race_df.copy()
temp_df.loc[temp_df['course_type'].isin(['I', 'O']), 'course_type'] = 'T'
temp_df = temp_df.groupby(['season', 'course_type'])['race_id'].count().to_frame().reset_index().rename(columns = {'race_id' : 'races'})
# temp_df.drop(temp_df[temp_df['course_type'].isin(['I', 'O'])].index, inplace = True)

In [ ]:
ax = sns.barplot(data = temp_df, x = 'season', y = 'races', hue = 'course_type', palette = ['#783f04', '#38761d'])

ax.bar_label(ax.containers[0])
ax.bar_label(ax.containers[1])

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

plt.title('Races & Season')
plt.figure(figsize = (10, 5))

In [ ]:
sub_dirt_df = (race_df.groupby(['season', 'dirt_condition'])['race_id'].count()
                                                                      .to_frame()
                                                                      .reset_index()
                                                                      .rename(columns = {"race_id" : 'counts'})
              )
sub_dirt_df['season_counts'] = sub_dirt_df.groupby('season')['counts'].transform('sum')
sub_dirt_df['seasonal_condition_rate'] = sub_dirt_df.apply(lambda x : round((x['counts'] / x ['season_counts']), 2), axis = 1)
sub_dirt_df = sub_dirt_df[sub_dirt_df['counts'] != 0]

sub_turf_df = race_df.copy()
sub_turf_df.loc[sub_turf_df['course_type'].isin(['I', 'O']), 'course_type'] = 'T'
sub_turf_df = (race_df.groupby(['season', 'turf_condition'])['race_id'].count()
                                                                       .to_frame()
                                                                       .reset_index()
                                                                       .rename(columns = {"race_id" : 'counts'})
              )
sub_turf_df = sub_turf_df[sub_turf_df['counts'] != 0]

In [ ]:
sub_dirt_df.head(2)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5), sharey = True)


sns.barplot(data = sub_dirt_df,
              x = 'season',
              y = 'counts',
             hue = 'dirt_condition',
             ax = ax[0],
             palette = dirt_palette
           )

ax[0].set_title('Seasonal Dirt Track Condition')
ax[0].set_ylabel('Races')
ax[0].legend().set_visible(False)
box = ax[0].get_position()
ax[0].set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[0].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))



sns.barplot(data = sub_turf_df,
              x = 'season',
              y = 'counts',
             hue = 'turf_condition',
             ax = ax[1],
           palette = turf_palette
           )

ax[1].set_title('Seasonal Turf Track Condition')
ax[1].set_ylabel(None)

# legend
box = ax[1].get_position()
ax[1].set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax[1].legend(loc = 'center left', bbox_to_anchor=(1, 0.5))

for i in range(2):
    for j in range(4):
        ax[i].bar_label(ax[i].containers[j])

In [ ]:
sns.lineplot(data = (sub_dirt_df
                     [sub_dirt_df['dirt_condition'].isin(['FT', 'SY'])][:]
                    ), 
            x = 'season', 
            y = 'seasonal_condition_rate', 
            hue = 'dirt_condition',
            palette = dirt_palette,
            alpha = 0.75)
# plt.gca().legend().set_visible(False)

plt.title('Seasonal Dirt Condition Rate : FT vs SY')

$SeasonalDirt(Turf)Condition Rate = \frac{Dirt Condition Races}{Total Dirt(Turf) Races}$

# 2. Jockey

In [ ]:
merged_df = start_df.merge(race_df,
                on = ['race_id'])
# merged_df.insert(0, 'race_id', merged_df.pop('race_id'))
merged_df.head(2)

## Jockeys, number of competitions, average ranking, average ranking per course type
### Summary : Use all data

1. The null hypothesis is that "the number of games and the order of participation have no relationship," and the results differ depending on how the data is prepared.
    1. ***Excluding the range exceeding 'IQR'**: Regarding the number of matches and the average ranking, **There is little correlation + the p-value value is also large**
    2. **Full data**: **The correlation coefficient is high + the p-value is also a value that can reject the null hypothesis.**
2. When each of the riders is placed on the scatterplot, the ranking of the riders is integrated into the mean for the number of participants, and then the correlation coefficient and p-value value are different.

- What can be said to be right?
    - There are a total of 162 players, and 29 players have played more than "Upper IQR Boundary: 82 games," accounting for about 17%. I don't think it's a negligible figure.

1. 귀무가설은 "경기 수와 출전 순위는 아무런 관계가 없다"이고, 데이터를 어떻게 준비하느냐에 따라 다른 결과가 나왔음.
    1. **`IQR`을 넘는 범위 제외** : 경기수와 평균 순위에 관해, **상관관계가 거의 없음 + p-value 값도 크게 나옴**
    2. **전체 데이터** : **상관계수가 높게 나옴 + p-value도 귀무가설을 기각할 수 있는 수치가 나옴**
2. 기수 각각을 산점도에 놨을 때 순위와, 출전수에 대해 기수들의 순위를 평균으로 통합한 다음 산점도에 놨을 때, 상관계수와 p-value값은 다르게 나옴

- 무엇이 맞다고 할 수 있을까?
    - 총 162명인데 `Upper IQR Boundary : 82경기` 이상 뛴 기수가 29명으로 약 17%를 차지한다. 무시할만한 수치는 아니라고 생각됨.

### Find Outliers

In [ ]:
jockey_df = merged_df.loc[:, ['race_id', 'jockey', 'course_type', 'dirt_condition', 'turf_condition', 'position_at_finish']]
jockey_df['num_of_competitions'] = jockey_df.groupby('jockey')['race_id'].transform('count')
jockey_df['avg_rank_by_jockey']= jockey_df.groupby(['jockey'])['position_at_finish'].transform('mean')
jockey_app_rank = jockey_df[['jockey', 'num_of_competitions', 'avg_rank_by_jockey']].drop_duplicates()
jockey_app_rank['jockey'].count()
jockey_app_rank.describe()

In [ ]:
sns.boxplot(data = jockey_app_rank, x = 'num_of_competitions')

plt.xlim([0, 80])
plt.title("num_of_competitions Data Except Outliers")

In [ ]:
# upper_boundary
x = jockey_app_rank['num_of_competitions']
q_75, q_25 = np.percentile(x, [75, 25])

iqr = q_75 - q_25
up_boundary = q_75 + 1.5 * iqr
up_boundary

In [ ]:
jockey_app_rank[jockey_app_rank['num_of_competitions'] >= up_boundary]['jockey'].count()

#### 1. num_of_competitions : Exclude outlier data beyond IQR

In [ ]:
jockey_app_rank_inner = jockey_app_rank[jockey_app_rank['num_of_competitions'] <= up_boundary][:]
jockey_app_rank_inner

In [ ]:
sns.regplot(data = jockey_app_rank_inner, x = 'num_of_competitions', y = 'avg_rank_by_jockey')
plt.title("without outlier : jockey and avg_rank")
r, p_value = stats.pearsonr(x = jockey_app_rank_inner['num_of_competitions'], y = jockey_app_rank_inner['avg_rank_by_jockey'])
print(f"corcoef = {r}, p-value = {p_value}")

In [ ]:

temp_df = jockey_app_rank_inner.groupby('num_of_competitions')['avg_rank_by_jockey'].mean().to_frame().reset_index().rename(columns = {'avg_rank_by_jockey' : 'avg_rank'})
sns.regplot(data = temp_df, x = 'num_of_competitions', y = 'avg_rank')
plt.title('without outlier : num_of_competitions and avg_rank')
r, p_value =stats.pearsonr(x = temp_df['num_of_competitions'], y = temp_df['avg_rank'])
print(f"corcoef = {r}, p-value = {p_value}")

#### 2. Plot without removing outlier data
- The total number of jockeys is 162, and the value of 'nunique()' is 58.

In [ ]:
sns.regplot(data = jockey_app_rank,
               x = 'num_of_competitions',
               y = 'avg_rank_by_jockey',
#                ci = None,
               scatter_kws = {'color' : 'green'})

plt.title("with outlier : jockey and avg_rank")
stats.pearsonr(x = jockey_app_rank['num_of_competitions'], y = jockey_app_rank['avg_rank_by_jockey'])

In [ ]:
dirt_appear_df = (jockey_df[jockey_df['course_type'] == 'D'].groupby('jockey')['race_id'].count()
                                                                                         .to_frame()
                                                                                         .reset_index()
                                                                                         .rename(columns = {'race_id' : 'dirt_num_of_competitions'}))
turf_appear_df = (jockey_df[jockey_df['course_type'].isin(['T', 'I', 'O'])].groupby('jockey')['race_id'].count()
                                                                                                     .to_frame()
                                                                                                     .reset_index()
                                                                                                     .rename(columns = {'race_id' : 'turf_num_of_competitions'}))

jockey_df = (jockey_df.merge(dirt_appear_df, how = 'left', on = 'jockey')
                      .merge(turf_appear_df, how = 'left', on = 'jockey'))
jockey_df.info()


In [ ]:
jockey_df['dirt_num_of_competitions'].fillna(0, inplace = True)
jockey_df['turf_num_of_competitions'].fillna(0, inplace = True)

jockey_df.info()

In [ ]:
# 1. 출전 수와 평균 순위
jockey_df['num_of_competitions'] = jockey_df.groupby('jockey')['race_id'].transform('nunique')
app_rank = (jockey_df.groupby('num_of_competitions')['position_at_finish'].mean()
                                                                 .to_frame()
                                                                 .reset_index()
                                                                 .rename(columns = {'position_at_finish' : 'avg_rank'}))

# 2. Dirt 출전 수와 평균 순위
dirt_app_rank = (jockey_df[jockey_df['course_type'] == 'D'].groupby('dirt_num_of_competitions')['position_at_finish']
                                                           .mean()
                                                           .to_frame()
                                                           .reset_index()
                                                           .rename(columns = {'position_at_finish' : 'dirt_avg_rank'}))

turf_app_rank = (jockey_df[jockey_df['course_type'].isin(['T', 'I', 'O'])].groupby('turf_num_of_competitions')['position_at_finish']
                                                                           .mean()
                                                                           .to_frame()
                                                                           .reset_index()
                                                                           .rename(columns = {'position_at_finish' : 'turf_avg_rank'}))

In [ ]:
def get_regplot(df, col1_name, col2_name, color = 'blue'):
    sns.regplot(data = df,
                x = col1_name,
                y = col2_name,
#                 ci = None,
                color = color,
                line_kws = {
#                             'color' : 'grey',
                            'lw' : 3
                },
                scatter_kws = {'s' : 10,
#                               'color' : 'grey'
                              })
    
    plt.ylim([3, 8])

def get_pearsonr(col1, col2):
    corrcoef, p_value = stats.pearsonr(x = col1,
                                       y = col2)
    print(f"corrcoef : {corrcoef}, p-value : {p_value}")
    
get_regplot(app_rank, 'num_of_competitions', 'avg_rank')
get_pearsonr(app_rank['num_of_competitions'], app_rank['avg_rank'])
plt.title("Total num_of_competitions and Average Rank", fontsize = 20)

In [ ]:
# sns.set_palette(sns.color_palette(dirt_colors))
get_regplot(dirt_app_rank, 'dirt_num_of_competitions', 'dirt_avg_rank', color = 'brown')
get_pearsonr(dirt_app_rank['dirt_num_of_competitions'], dirt_app_rank['dirt_avg_rank'])
plt.title("Dirt : num_of_competitions and Average Rank", fontsize = 20)

In [ ]:
turf_palette
get_regplot(turf_app_rank, 'turf_num_of_competitions', 'turf_avg_rank', color = 'green')
get_pearsonr(turf_app_rank['turf_num_of_competitions'], turf_app_rank['turf_avg_rank'])
plt.title("Turf : num_of_competitions and Average Rank", fontsize = 20)

### View the stadium
- [link](https://www.kaggle.com/code/iamleonie/big-data-derby-step-by-step-analysis#Missing-Data) 
- I didn't think I could draw the stadium by looking at the longitude and latitude data. I'll just look at the idea and implement it myself

In [ ]:
track = start_df['track_id'].unique()
total_df = race_df.merge(track_df, on = ['track_id', 'race_date', 'race_number'])

In [ ]:
track_lst = race_df['track_id'].unique()
track_lst

In [ ]:
fig, ax = plt.subplots(3, 1, figsize = (15, 45))
track_palette = {'D' : '#744700',
        'T' : '#38761d',
        'I' : '#91cd76',
        'O' : '#91cd76', 
        'M' : '#d9ead3'}

for i, v in enumerate(track_lst):
    temp_df = (
                total_df[(total_df['track_id'] == v) & 
                        (total_df['trakus_index'].isin(list(range(0, 200)))) &
                        (total_df['program_number'].isin(['1', '2', '3']))
                       ]
                        [['course_type', 'latitude', 'longitude']]
                )

    sns.scatterplot(data = temp_df,
                       x = 'latitude',
                       y = 'longitude',
                       hue = 'course_type',
                        ax = ax[i],
                   palette = track_palette)
    
    ax[i].set_title(f'Track : {v}')

# 3. business analysis
- Only analyzed the largest 'Stakes' race

- Race distance ranges from 3630 to 10560 (unit: feet)
- `run_up_distance` : `9 ~ 246`
- 'trakus_index' is well stamped in chronological order.

In [ ]:
race_track = track_df.merge(race_df, on = ['race_id', 'track_id', 'race_date', 'race_number'])
# race_track.insert(0, 'race_id', race_track.pop('race_id'))
race_track.drop(['track_id', 'race_date', 'race_number'], axis = 1, inplace = True)
race_track.head(2)

In [ ]:
stakes = race_track[race_track['race_type'] == 'STK'][:] # 경기 거리는 3630 ~ 10560까지 있다
stk_dirt = stakes[stakes['course_type'] == 'D'][:].drop('turf_condition', axis = 1)

## Selecting and Analyzing a Race
- STK will catch any competition and measure the speed of each driver's Trakus Index
- Speed can be obtained because the interval between Trakus indexes is stated to be 0.25 seconds
- Apply 'haversine' library

In [ ]:
# trakus_index로 계산
race_name = 'SAR_2019-08-10_8'
test_race = stk_dirt[stk_dirt['race_id'] == race_name][:]

test_race['next_lat'] = test_race.sort_values(by = ['program_number', 'trakus_index']).groupby('program_number')['latitude'].shift(-1)
test_race['next_lon'] = test_race.sort_values(by = ['program_number', 'trakus_index']).groupby('program_number')['longitude'].shift(-1)

test_race.loc[test_race['next_lat'].notnull(), 'moved'] = (test_race.apply
                                                                           (lambda x : haversine((x['latitude'], x['longitude']), 
                                                                                                 (x['next_lat'], x['next_lon']), 
                                                                                                  unit = 'ft'), 
                                                                            axis = 1)
                                                            )

test_race['velocity_ft/s'] = test_race['moved'] / 0.25 # feet per second
test_race['moved_km'] = test_race['moved'].apply(lambda x : x / 3281)
test_race['velocity_km/h'] = test_race['velocity_ft/s'].apply(lambda x : x / 3281 * 3600)
test_race['elapsed_time'] = (test_race['trakus_index'] - 1) * 0.25
test_race['total_distance'] = test_race.sort_values(by = ['program_number', 'trakus_index']).groupby('program_number')['moved'].cumsum()


In [ ]:
test_race.describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (30, 10))

sns.lineplot(data = test_race,
            x = 'elapsed_time',
            y = 'velocity_km/h',
            hue = 'program_number',
            ax = ax[0])
plt.title(f"race {'SAR_2019-08-10_8'} : Time and Velocity")

ax[0].legend().set_visible(False)

sns.lineplot(data = test_race,
            x = 'elapsed_time',
            y = 'total_distance',
            hue = 'program_number',
            ax = ax[1])

plt.title(f"race {'SAR_2019-08-10_8'} : Time and Distance")

In [ ]:
# 30초 ~ 50초 즘에서 나오는 속도가 일정하게 유지되는 구간이 궁금하다

data = test_race[test_race['elapsed_time'].between(31, 50, inclusive = "both")][:]
else_data = test_race[test_race['elapsed_time'].between(31, 50, inclusive = "both") == False][:]

sns.set_palette(turf_palette)
sns.scatterplot(data = data,
               x = 'latitude',
               y = 'longitude',
               color = 'red')

sns.scatterplot(data = else_data,
               x = 'latitude',
               y = 'longitude',
               color = 'grey')


## Get speed from the stakes table
- Apply the above process for individual races to the entire Stake Races

In [ ]:
stakes_raw = race_track[race_track['race_type'] == 'STK'][:] 
stakes_raw.drop(['distance_id', 'race_type', 'month', 'season'], axis = 1, inplace = True)
stakes_raw.head(2)

In [ ]:
stakes_raw['next_lat'] = stakes_raw.sort_values(by = ['race_id', 'program_number', 'trakus_index']).groupby(['race_id', 'program_number'])['latitude'].shift(-1)
stakes_raw['next_lon'] = stakes_raw.sort_values(by = ['race_id', 'program_number', 'trakus_index']).groupby(['race_id', 'program_number'])['longitude'].shift(-1)

stakes_raw.loc[stakes_raw['next_lat'].notnull(), 'moved_ft'] = (stakes_raw.apply
                                                                           (lambda x : haversine((x['latitude'], x['longitude']), (x['next_lat'], x['next_lon']), 
                                                                                                  unit = 'ft'
                                                                                                ),
                                                                            axis = 1)
                                                            )
stakes_raw['velocity_ft'] = stakes_raw['moved_ft'] / 0.25 # feet per second
stakes_raw.loc[stakes_raw['next_lat'].notnull(), 'moved_km'] = (stakes_raw.apply
                                                                           (lambda x : haversine((x['latitude'], x['longitude']), (x['next_lat'], x['next_lon']), 
                                                                                                  unit = 'km'
                                                                                                ),
                                                                            axis = 1)
                                                            )
stakes_raw['velocity_km/h'] = stakes_raw['moved_km'].apply(lambda x : x / 0.25 * 3600)
stakes_raw['elapsed_time'] = (stakes_raw['trakus_index'] - 1) * 0.25
stakes_raw['player_cumulative_moved'] = stakes_raw.sort_values(by = ['race_id', 'program_number', 'trakus_index']).groupby(['race_id', 'program_number'])['moved_km'].cumsum()

stakes_raw

In [ ]:
stakes_raw.describe()

## Clean up outlier data
- The maximum instantaneous speed taken by a horse is said to be 88 km/h.
- [Related Information](http://www.speedofanimals.com/animals/horse)
- Any value larger than $89.584 km/h$ is treated as an outlier, excluding races with outliers
    - The latitude and longitude data represented up to the sixth decimal place have an error of $0.11 m = 0.00011 km$.
    - And the error speed can have is $0.00011 \times 4 \times 3600$ = $1.584 km/h$. (location is taken every 0.25 seconds)
    - I don't know if the calculation is correct. Welcome criticism.

#### 참고 : 위도, 경도 데이터의 소수점 갯수와 그에 따른 오차
- [관련 링크](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude)
- 내가 제대로 이해한 게 맞다면
    - 일반적으로 위도`latitude` 1도는 111111m를 덮는다고 알려져 있다. $\frac{10^7}{90}$
    - 경도`longitude` 또한 위도와 비슷하거나 작다. 작아지는 이유는 극으로 갈수록 폭이 좁아지기 때문임
    - 따라서 소수점 6째자리까지 표현된 위도, 경도 데이터가 갖는 오차는 $0.11m = 0.00011km$ 정도이다.
    - 그리고 속도가 가질 수 있는 오차는 $0.00011 \times 4 \times 3600$ = $1.584km/h$이다. (위치는 0.25초마다 찍힘)
    - 계산이 맞나 모르겠음. 비판 환영합니다..
- 따라서 $89.584km/h$보다 큰 값은 모두 이상치 취급하고, 이상치가 들어간 레이스는 제외하겠음

In [ ]:
# outliers = stakes[stakes['velocity_km/h'] >= 89.584][:] 
outliers = stakes_raw[stakes_raw['velocity_km/h'] >= 89.584][:] 

outliers.shape # (218, 24)
outliers_races = outliers['race_id'].unique() # 64개의 레이스에 이상치 데이터가 있음 / Stakes 전체 253개

In [ ]:
stakes = stakes_raw.drop(stakes_raw[stakes_raw['race_id'].isin(outliers_races)].index)
stakes['race_id'].nunique() # 189

### Dirt vs Turf

### Summary
1. The average speed of `Turf` was higher than that of `Dirt` ($67.97 km/h$ vs $66.30 km/h$)
    - Other variables need to be investigated further. The streets of Gyeongju, the weight of the horse.
2. In the case of stadium condition (`dirt_condition` and `turf_condition`), the average speed decreases in common as the ground holds moisture
    - Compared to the best and worst conditions, the average speed reduction of `Turf` was higher ($6.7%), 'Dirt': $3.2%)
3. There was little difference according to Inner Turf, Turf, and Outer Turf.
    - I think it would be okay to tie it with 'Turf' as long as the nature of the track itself is laid down.

1. Average speed depending on track condition: There seems to be no difference

In [ ]:
ax = stakes.groupby('course_type')['velocity_km/h'].mean().plot(kind = 'bar', color = ['brown'] + ['green'] * 3)
ax.bar_label(ax.containers[0])
plt.title("course_type vs Average Speed")
plt.ylabel('average speed')

2. Average speed depending on track and track conditions

In [ ]:
# 1. Dirt
dirt = stakes[stakes['course_type'] == 'D'].groupby('dirt_condition')['velocity_km/h'].mean().to_frame().reset_index()

plt.figure(figsize = (10, 5))
ax = sns.barplot(data = dirt,
           x = 'dirt_condition',
           y = 'velocity_km/h',
           palette = dirt_palette)
ax.bar_label(ax.containers[0])
plt.title("Dirt : Track Condition and Average Speed")

In [ ]:
# 2. turf
turf = stakes[stakes['course_type'].isin(['T', 'I', 'O'])].groupby(['course_type', 'turf_condition'])['velocity_km/h'].mean().to_frame().reset_index()
tio = stakes[stakes['course_type'].isin(['T', 'I', 'O'])].groupby(['course_type'])['velocity_km/h'].mean().to_frame().reset_index()
fig, ax = plt.subplots(1, 2, figsize = (20, 7))

sns.barplot(data = tio,
           x = 'course_type',
           y = 'velocity_km/h',
           color = 'green',
           ax = ax[0])
ax[0].set_title('turf position and velocity')
ax[0].bar_label(ax[0].containers[0])


sns.barplot(data = turf,
           x = 'course_type',
           y = 'velocity_km/h',
           hue = 'turf_condition',
           palette = turf_palette,
           ax = ax[1])
ax[1].set_title('turf condition and velocity')

box = ax[1].get_position()
ax[1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[1].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

In [ ]:
turf = stakes[stakes['course_type'].isin(['T', 'I', 'O'])][:]
turf.loc[turf['course_type'].isin(['I', 'O']), 'course_type'] = 'T'
turf = turf.groupby(['course_type', 'turf_condition'])['velocity_km/h'].mean().to_frame().reset_index()

In [ ]:
# 시각화는 Dirt와 동시에 보자
fig, ax = plt.subplots(1, 2, figsize = (16, 7), sharey = True)

sns.barplot(data = turf, 
            x = 'turf_condition', 
            y = 'velocity_km/h', 
            ax = ax[0])
ax[0].set_title("Turf : Track Condition and Average Speed")


sns.barplot(data = dirt,
           x = 'dirt_condition',
           y = 'velocity_km/h',
           palette = dirt_palette,
           ax = ax[1])
ax[1].set_title("Dirt : Track Condition and Average Speed")

for i in range(2):
    for j in ['left', 'top', 'right']:
        ax[i].bar_label(ax[i].containers[0])
        ax[i].spines[j].set_visible(False)
        ax[i].get_yaxis().set_visible(False)

In [ ]:
stakes.loc[stakes['course_type'].isin(['I', 'O']), 'course_type'] = 'T'
plt.figure(figsize = (10, 7))
stakes.groupby('course_type')['velocity_km/h'].mean().plot(kind = 'bar', color = ['brown', 'green'])
plt.gca().bar_label(plt.gca().containers[0])
for j in ['left', 'top', 'right']:
    plt.gca().spines[j].set_visible(False)

plt.yticks(ticks = [])

plt.title("Course Type and Average Speed")


In [ ]:
stakes = stakes[['race_id', 'program_number', 'trakus_index', 'latitude', 'longitude', 'next_lat', 'next_lon',
                'course_type', 'dirt_condition', 'turf_condition', 'run_up_distance', 'moved_km', 'velocity_km/h', 'elapsed_time']]
# stakes['track_id'] = stakes['race_id'].apply(lambda x : x[0:3])
stakes = stakes.merge(start_df, on = ['race_id', 'program_number'])
stakes.insert(1, 'track_id', stakes.pop('track_id'))
stakes.loc[stakes['course_type'].isin(['I', 'O']), 'course_type'] = 'T'

### Condition 2 Weight and Average Speed

#### Summary
- Contrary to the conventional wisdom that the heavier the weight, the lower the average speed, **did not show a distinct tendency.**
- Rather - On the contrary, **The above once again shows that you can run faster on the Turf**
- The average speed according to the average weight of each competition also does not have a clear trend

In [ ]:
sns.histplot(data = stakes,
            x = 'weight_carried',
#              kde = True,
            bins = 20)
plt.title("weight histogram")

In [ ]:
temp_df = stakes[['race_id', 'program_number', 'course_type', 'dirt_condition', 'turf_condition', 'weight_carried', 'velocity_km/h', 'elapsed_time', 'position_at_finish']]
temp_df = temp_df[temp_df['weight_carried'].between(110, 130, inclusive = 'both')][:]
# print(temp_df.head(2))
temp_df = temp_df.groupby(['weight_carried', 'course_type'])['velocity_km/h'].agg(['mean', 'count']).dropna().reset_index()

sns.lineplot(data = temp_df,
           x = 'weight_carried',
           y = 'mean',
           hue = 'course_type',
          palette = [dirt_palette[0], turf_palette[0]])

plt.title('weight and average_speed')
plt.ylabel("average speed")

## 

### Average weight and average speed

In [ ]:
temp_df = stakes[['race_id', 'program_number', 'course_type', 'velocity_km/h', 'weight_carried']]
temp_df.head(2)

In [ ]:
temp_df.loc[:, 'race_avg_weight'] = temp_df.groupby('race_id')['weight_carried'].transform('mean')
temp_df.loc[:, 'race_avg_speed'] = temp_df.groupby('race_id')['velocity_km/h'].transform('mean')
temp_df.loc[:, 'player_avg_speed'] = temp_df.groupby(['race_id', 'program_number'])['velocity_km/h'].transform('mean')
temp_df.head(2)

In [ ]:
temp_df2 = temp_df.groupby(['course_type', 'race_avg_weight'])['race_avg_speed'].mean()
temp_df2

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 6))
sns.regplot(data = temp_df[temp_df['course_type'] == 'D'][:], 
             x = 'race_avg_weight', 
             y = 'race_avg_speed', 
             color = dirt_palette[0],
#              ci = True,
             ax = ax[0]
            )
ax[0].set_title('Dirt : Average Weight and Average Speed')


sns.regplot(data = temp_df[temp_df['course_type'] == 'T'][:], 
             x = 'race_avg_weight', 
             y = 'race_avg_speed', 
             color = 'green',
#             ci = True,
             ax = ax[1]
            )
ax[1].set_title('Turf : Average Weight and Average Speed')


### Condition 3: Average speed according to total mileage

#### Summary
1. Naturally, **The longer the distance, the lower the average speed**
2. **The mileage of each race** is not continuous and **discrete** (now I know this)
    - Of the 1991 data excluding the hurdle race in `race_df`, there are 16 unique values of the mileage
    - If you come to 'stakes', the race has a total of 189 data and 14 unique values for the mileage
3. It seems difficult to explain the reduction depending on the track condition except for the best management condition

In [ ]:
temp_df = stakes[['race_id', 'course_type', 'dirt_condition', 'turf_condition', 'program_number', 'moved_km', 'velocity_km/h', 'elapsed_time']]
temp_df = temp_df.merge(race_df[['race_id', 'distance_feet']], on = 'race_id')
temp_df['race_distance_km'] = temp_df['distance_feet'].apply(lambda x : x / 3281)
temp_df = temp_df.drop('distance_feet', axis = 1)
temp_df['race_avg_speed_km/h'] = temp_df.groupby(['race_id'])['velocity_km/h'].transform('mean')

In [ ]:
temp_df = temp_df[['race_id', 'course_type', 'dirt_condition', 'turf_condition', 'race_distance_km', 'race_avg_speed_km/h']].drop_duplicates()

# fig, ax = plt.subplots(1, 2, figsize = (16, 9), sharex = True, sharey = True)

sns.lmplot(data = temp_df[temp_df['course_type'] == 'D'][:], 
            x = 'race_distance_km', 
            y = 'race_avg_speed_km/h', 
            hue = 'dirt_condition',
            palette = dirt_palette,
#            ax = ax[0],
           ci = False
          )
plt.title("Dirt : Race Distance and Average Speed")
# ax[0].set_title("Dirt : Race Distance and Average Speed")

In [ ]:
sns.lmplot(data = temp_df[temp_df['course_type'] == 'T'][:], 
            x = 'race_distance_km', 
            y = 'race_avg_speed_km/h', 
            hue = 'turf_condition',
            palette = turf_palette,
#            ax = ax[1],
           ci = False
          )
# ax[1].set_title("Turf : Race Distance and Average Speed")
plt.title("Turf : Race Distance and Average Speed")

In [ ]:
sns.histplot(temp_df['race_distance_km'], bins = 14)

### Condition 4: Average speed and average distance depending on the condition of the stadium and the ground

#### 1. In terms of the stadium alone, the average speed is **BEL > SAR > AQU**.
1. In `BEL` and `SAR`, relatively many `Turf` games were held & there were many games that maintained the bottom condition well
2. On the other hand, `AQU` had a higher proportion of `Dirt` games than the other two stadiums, and there were many games that did not maintain the best floor condition

#### 2. Meanwhile, there is a question about `AQU - DIRT - FT`
- Measured lower average speed despite shorter mileage compared to the other two stadiums
- Not even unusual cases with a small number of data. 


In [ ]:
race_df['race_distance_km'] = race_df['distance_feet'].apply(lambda x : x / 3281)
stakes = stakes.merge(race_df[['race_id', 'distance_feet', 'race_distance_km']], on = 'race_id')
stadium_df = stakes[['race_id', 'track_id', 'course_type', 'dirt_condition', 'turf_condition', 'race_distance_km', 'velocity_km/h']]


In [ ]:
# 중복 제거
no_dup_stdium = stadium_df[['race_id', 'track_id', 'course_type', 'dirt_condition', 'turf_condition', 'race_distance_km']].drop_duplicates()

dirt_stdium = no_dup_stdium[no_dup_stdium['course_type'] == 'D'][:]
turf_stdium = no_dup_stdium[no_dup_stdium['course_type'] == 'T'][:]

In [ ]:
id_dist = dict(x = 'track_id', y = 'race_distance_km')
''
fig, ax = plt.subplots(2, 2, figsize = (12, 12), sharey = True)

fig.suptitle("Stakes Race : Stadium, Track Condition, Distance")

sns.swarmplot(data = no_dup_stdium, **id_dist, ax = ax[0][0], palette = 'tab10')
sns.swarmplot(data = no_dup_stdium, **id_dist, hue = 'course_type', palette = dt_palette, alpha = 0.75, ax = ax[0][1])
# ax[0][1].ylabel.set_visible(False)
sns.swarmplot(data = dirt_stdium, **id_dist, hue = 'dirt_condition', palette = dirt_palette, ax = ax[1][0])
sns.swarmplot(data = turf_stdium, **id_dist, hue = 'turf_condition', palette = turf_palette, ax = ax[1][1])
# ax[1][1].ylabel.set_visible(False)


In [ ]:
stadium_df = stakes[['race_id', 'track_id', 'course_type', 'dirt_condition', 'turf_condition', 'velocity_km/h', 'moved_km']]
stadium_df = stadium_df.merge(race_df[['race_id','race_distance_km']], on = 'race_id')
stadium_df

In [ ]:
dist_speed = stadium_df.groupby(['track_id'])[['velocity_km/h', 'race_distance_km']].mean().reset_index().dropna()
track_dist_speed = stadium_df.groupby(['track_id', 'course_type'])[['velocity_km/h', 'race_distance_km']].mean().reset_index().dropna()
dirt_cond_dist_speed = stadium_df[stadium_df['course_type'] == 'D'].groupby(['track_id', 'dirt_condition'])[['velocity_km/h', 'race_distance_km']].mean().reset_index().dropna()
turf_cond_dist_speed = stadium_df[stadium_df['course_type'] == 'T'].groupby(['track_id', 'turf_condition'])[['velocity_km/h', 'race_distance_km']].mean().reset_index().dropna()

In [ ]:

fig, ax = plt.subplots(4, 2, figsize = (12, 15))
fig.suptitle("DISTANCE AND VELOCITY")
sns.set_style("ticks",{'axes.grid' : True})
sns.barplot(
                 data = dist_speed, 
                 x = 'track_id', 
                 y = 'velocity_km/h',
                 palette = 'tab10',
                 ax = ax[0][0]
)

ax[0][0].set_ylim([65, 70])
ax[0][0].set_title("Average Speed")

sns.barplot(
                 data = dist_speed, 
                 x = 'track_id', 
                 y = 'race_distance_km',
                 palette = 'tab10',
                 ax = ax[0][1]
)

ax[0][1].set_ylim([1.6, 1.8])
ax[0][1].set_title("Average Race Distance")

sns.barplot(
                 data = track_dist_speed, 
                 x = 'track_id', 
                 y = 'velocity_km/h',
                 hue = 'course_type',
                 palette = dt_palette,
                 ax = ax[1][0]
)
sns.barplot(
                 data = track_dist_speed, 
                 x = 'track_id', 
                 y = 'race_distance_km',
                 hue = 'course_type',
                 palette = dt_palette,
                 ax = ax[1][1]
)
ax[1][0].legend().set_visible(False)
box = ax[1][1].get_position()
ax[1][1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[1][1].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

sns.barplot(
                 data = dirt_cond_dist_speed, 
                 x = 'track_id', 
                 y = 'velocity_km/h',
                 hue = 'dirt_condition',
                 palette = dirt_palette,
                 ax = ax[2][0]
)
sns.barplot(
                 data = dirt_cond_dist_speed, 
                 x = 'track_id', 
                 y = 'race_distance_km',
                 hue = 'dirt_condition',
                 palette = dirt_palette,
                 ax = ax[2][1]
)

ax[2][0].legend().set_visible(False)
box = ax[2][1].get_position()
ax[2][1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[2][1].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

sns.barplot(
                 data = turf_cond_dist_speed, 
                 x = 'track_id', 
                 y = 'velocity_km/h',
                 hue = 'turf_condition',
                 palette = turf_palette,
                 ax = ax[3][0]
)
sns.barplot(
                 data = turf_cond_dist_speed, 
                 x = 'track_id', 
                 y = 'race_distance_km',
                 hue = 'turf_condition',
                 palette = turf_palette,
                 ax = ax[3][1]
)

ax[3][0].legend().set_visible(False)
box = ax[3][1].get_position()
ax[3][1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax[3][1].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

plt.setp([ax[i][0] for i in range(4)], ylim = (60, 70))
plt.setp([ax[i][1] for i in range(4)], ylim = (1.25, 2.1))

# 값 표시
# for i in range(4):
#     for j in range(2):
#         for k in range(len(ax[i][j].containers)):
#             ax[i][j].bar_label(ax[i][j].containers[k])

fig.tight_layout()

In [ ]:
race_counts = stadium_df[['race_id', 'track_id', 'course_type', 'dirt_condition', 'turf_condition', 'race_distance_km']].drop_duplicates()
race_counts.head(3)

In [ ]:
dirt= pd.pivot_table(race_counts[race_counts['course_type'] == 'D'],
              index = 'track_id',
              columns = ['course_type', 'dirt_condition'],
              values = 'race_id',
              aggfunc = 'count')

dirt.columns = ['D_FT', 'D_GD', 'D_MY', 'D_SY']
dirt= dirt.reset_index()

turf= pd.pivot_table(race_counts[race_counts['course_type'] == 'T'],
              index = 'track_id',
              columns = ['course_type', 'turf_condition'],
              values = 'race_id',
              aggfunc = 'count')
turf.columns = ['T_FM', 'T_GD', 'T_YL', 'T_SF']
turf= turf.reset_index()

merged = dirt.merge(turf, on = 'track_id')

merged

In [ ]:
merged_palette = dirt_palette + turf_palette

fig, ax = plt.subplots(figsize = (12, 6))

btm = np.zeros(len(merged))

for i, val in enumerate(merged.columns[1:]):
    ax.bar(merged['track_id'], merged[val], bottom = btm, label = val, color = merged_palette[i])
    btm += np.array(merged[val])
    
plt.title("Stakes : The Number Of Races and Ground Condition")

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

# ax.bar_label(ax.containers[0])
# ax.bar_label(ax.containers[4])

### Condition 5. `run_up_distance`

#### Summary
- **'trakus_index' is not exactly the beginning to end of the game**
- When they left the gate ~ Recorded even after the competition ends
- Therefore, the average speed obtained above is not an accurate value: an integral calculation is likely to be necessary to obtain a more accurate value.

- [Description 1](https://paulickreport.com/horseplayers-category/thoroughbred-idea-foundation-run-up-made-final-time-misleading-in-jessamine/)
- [Description 2](https://www.youtube.com/watch?v=9xx-0LvEAk8)
    - If you look at `2:00`, there are two intuitive examples
- The total distance of the race is from gate to destination.
- However, measuring time in the horse race is not when you leave the gate, but at a certain point, which is a certain from the gate.
- The distance from the gate to the point where the time starts is called `run_up_distance`.

**Is 'trakus_index' measured from the time you started leaving 'run_up_distance'? Or is it being measured from the point of departure from the gate?**
- How can I get this?
    - Find the cumulative distance
    - Is this value greater than `race_distance`? Or is it smaller than `race_distance` and larger than `race_distance` - `run_up_distance`?
    - I think we can compare these two

In [ ]:
stakes['run_up_distance_km'] = stakes['run_up_distance'].apply(lambda x : x / 3281)
stakes['without_run_up_km'] = stakes['race_distance_km'] - stakes['run_up_distance_km']
stakes = stakes.sort_values(by = 'trakus_index', ascending = True)
stakes['total_moved_km'] = (stakes.groupby(['race_id', 'program_number'])['moved_km']
                                  .cumsum()
                           )
test = stakes.sample(6)['race_id'].values

In [ ]:
fig, ax = plt.subplots(len(test), 3, figsize = (18, 54))
trakus_idx_min = 0
trakus_idx_max = 10


for i in range(len(test)):
    sample = stakes[stakes['race_id'] == test[i]][:]
    sns.lineplot(
                data = sample[sample['trakus_index'].between(trakus_idx_min, trakus_idx_max, inclusive = 'both')],
                 x = 'trakus_index', 
                 y = 'velocity_km/h', 
                 hue = 'program_number', 
                 ax = ax[i][0],
                palette = 'tab10'
    )  
    ax[i][0].legend().set_visible(False)
#     ax[i][0].axvline(x = sample.loc[sample.index[0], 'run_up_distance_km'], color = 'black', linestyle = '--')
#     ax[i][0].text(10.1,0,'blah',rotation=90)
    sns.lineplot(
                    data = sample[sample['trakus_index'].between(trakus_idx_min, trakus_idx_max, inclusive = 'both')],
                     x = 'total_moved_km', 
                     y = 'velocity_km/h', 
                     hue = 'program_number', 
                     ax = ax[i][1],
                    palette = 'tab10'
        )
    ax[i][1].axvline(x = sample.loc[sample.index[0], 'run_up_distance_km'], color = 'black', linestyle = '--')
    ax[i][1].legend().set_visible(False)

    sns.scatterplot(data = sample[sample['trakus_index'].between(trakus_idx_min, trakus_idx_max, inclusive = 'both')],
                    x = 'latitude' ,
                    y = 'longitude', 
                    hue = 'program_number', 
                    ax = ax[i][2],
                    palette = 'tab10'
                   )
    box = ax[i][2].get_position()
    ax[i][2].set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax[i][2].legend(loc = 'center left', bbox_to_anchor = (1, 0.5))
    ax[i][2].grid(False)
    
fig.tight_layout()
# fig.suptitle(f'Observe {trakus_idx_max * 0.25}sec after the start of the match')


### Looks like coordinates are starting to take place as soon as you leave the gate
- If the speed starts to be taken after `run_up_distance`, the measurement will start at some speed.
- Each row represents a sample competition, and the second graph is a plot showing the cumulative travel distance and the speed at that time. The black dotted line corresponds to `run_up_distance`.

- On the other hand, compare the total distance taken in the above-mentioned latitude/longitude coordinates with the average value of the actual distance for each horse

In [ ]:
comp = stakes[['race_id', 'trakus_index', 'program_number', 'race_distance_km', 'run_up_distance_km', 'without_run_up_km', 'total_moved_km']]
comp2 = (comp.groupby('race_id')['trakus_index'].max()-1).to_frame().reset_index()
merged = comp.merge(comp2, on = ['race_id', 'trakus_index'])

merged[['avg_moved_km', 'avg_without_run_up']] = merged.groupby('race_id')[['total_moved_km', 'without_run_up_km']].transform('mean')
merged = merged[['race_id', 'race_distance_km', 'avg_moved_km', 'avg_without_run_up']].drop_duplicates()
(merged[['race_distance_km', 'avg_moved_km', 'avg_without_run_up']].rename(columns = {'race_distance_km' : 'known_distance',
                                                                                     'avg_moved_km' : 'actual_moved',
                                                                                     'avg_withpout_run_up' : 'actual_moved_without_run_up'})
                                                                     .mean()
                                                                     .plot(kind = 'bar', color = ['red', 'blue', 'green'])
)
plt.xticks(rotation = 0)
plt.ylabel('distance')
plt.gca().bar_label(plt.gca().containers[0])
plt.title("Known Distance vs Actual Moved")

- `trakus_index` does not accurately measure the start - end of a competition.
- This is because the distance of movement (red bar) specified in the game is about 160 meters different from the actual distance of movement (blue bar).
- On the other hand, on average, the value excluding `run_up_distance` has only an error of about 20m with the travel distance specified in the game.

### So to put it together with what we talked about above
1. **The time when the actual race is measured and the time when 'trakus_index' is measured are different.** The former is included in the latter.
2. If only the recorded game is to be measured, the accumulated moving distance and average speed obtained above for the reason of 1.
- However, it is not necessary to improve each instantaneous speed measured by 'trakus_index'.
    - The time when the first word passes by `run_up_distance`
    - Time each horse passes through the end position (`race_distance`)
- If you only get the above two, you'll be able to get the average horse speed in each game
- I thought of a heuristic approach first, but **There is an error with the actual ranking due to an interval of 0.25 seconds**
- So if you want to use the data to **exact point**, you'd be right to **use the integration**

-----------------------------------

#### Heuristic approach: failure

#### summary
- Use `trakus_index` to record the start and finish of the competition (**0.25 seconds of error**)
- Even if it was heuristic, there was an assumption that the average speed of the recorded interval would be ranked in the order of high speed
    - It wasn't like that in reality
- The conclusion that the race needs to save continuity even if it is not continuous data, and integration will be necessary for this

- Find the average speed for each race
- It looks like the best way to find integration, but I'll go with heuristics
    - How many seconds after the start of the game, I thought there would be data on whether it passed `run_up_distance` so I approached like this.
    - I don't know if there will be a calculation cost issue.
- Heuristics: The data taken at trackus: n is measured by the distance and speed traveled to n -> n+1
    - Therefore, you can select the smallest value among the values of `trakus_index` greater than `run_up_distance`

In [ ]:
# 특정 레이스부터 보겠음
sample_race = stakes['race_id'].sample(1).values[0]
sample_run = stakes[stakes['race_id'] == sample_race][:]

In [ ]:
sample_run.columns

In [ ]:
sample_run = sample_run[['program_number', 'trakus_index', 'elapsed_time', 'total_moved_km', 'velocity_km/h', 'race_distance_km', 'run_up_distance_km']]
sample_run['elapsed_time'] = sample_run['elapsed_time'] + 0.25
sample_run

In [ ]:
find_start_time = (sample_run[sample_run['total_moved_km'] >= sample_run['run_up_distance_km']]
                                                                         .sort_values(by = 'trakus_index'))
sample_run['start_time'] = find_start_time.loc[find_start_time.index[0], 'elapsed_time']

#### It can be said that the game started between 0.25 seconds before `start_time` and `start_time`
- I set it up like this to make sure that the game area is clear
- The end of the competition is also set to the point at which the cumulative travel distance exceeds `race_distance_km`

In [ ]:
find_end_time = (sample_run[sample_run['total_moved_km'] >= sample_run['race_distance_km']].sort_values(by = 'trakus_index')
                                                                                                           .groupby('program_number')['elapsed_time']
                                                                                                            .min())
sample_run = sample_run.merge(find_end_time.to_frame().rename(columns = {'elapsed_time' : 'finish_time'}), on = 'program_number')
sample_run['avg_speed'] = (sample_run['race_distance_km'] - sample_run['run_up_distance_km']) / (sample_run['finish_time'] - sample_run['start_time']) * 3600
result = sample_run[['program_number', 'avg_speed']].drop_duplicates()

In [ ]:
result = result.merge(start_df[start_df['race_id'] == sample_race][['program_number', 'position_at_finish']], on = 'program_number')

In [ ]:
result.sort_values(by = 'avg_speed', ascending = False)

#### Conclusion: The method of measuring the average speed after heuristic selection of the game time is wrong
- Therefore, it is important to select the exact time of the game, but I think it will be possible using integration (although this is not a completely accurate value)
- However, I can't proceed with this process because I don't have time and I want to try something else

----------------------------------


#### AQU_DIRT_FT
- About 'DIRT_FT': Why is the average speed slow even though the average race length is short?
#### Summary
- I tried 2 criteria, but neither of them got a satisfactory explanation.
1. The **average weight** of `AQU` was not particularly heavier.
2. The tendency of the average speed to decrease with **number of participants** is weak. However, races with a large number of participants are often long.
3. If you fix both the floor condition and length and look at the trend according to the number of participants, you can see a tendency to decrease the speed from 5 to 8 participants, but it cannot be said that always.

In [ ]:
dirt_ft = stakes[(stakes['course_type'] == 'D') & 
                (stakes['dirt_condition'] == 'FT')][:]
dirt_ft['elapsed_time'] += 0.25
dirt_ft = dirt_ft.drop(['latitude', 'longitude', 'next_lat', 'next_lon', 'course_type', 'dirt_condition', 'turf_condition',
             'race_date'], axis = 1)

In [ ]:
# 1. 위에서도 봤지만 다시 한번 평균 거리와 평균 속도
temp = dirt_ft.groupby('track_id')['velocity_km/h', 'race_distance_km'].mean().reset_index()

fig, ax = plt.subplots(1, 2, figsize = (10, 5))

fig.suptitle("Average Distance and Velocity")

sns.barplot(data = temp, x = 'track_id', y = 'race_distance_km', palette = 'tab10', ax = ax[0])
ax[0].bar_label(ax[0].containers[0])
ax[0].set_ylim(1.5, 1.8)

sns.barplot(data = temp, x = 'track_id', y = 'velocity_km/h', palette = 'tab10', ax = ax[1])
ax[1].bar_label(ax[1].containers[0])
ax[1].set_ylim(65, 70)

In [ ]:
# AQU = dirt_ft[dirt_ft['track_id'] == 'AQU'][:]
# BEL = dirt_ft[dirt_ft['track_id'] == 'BEL'][:]
# SAR = dirt_ft[dirt_ft['track_id'] == 'SAR'][:]

#### Approach 1. Average Weight
- Although no clear trend was found between weight and speed above, it can be inferred that there may be a relationship.
- Conclusion) No. There is little difference in average weight.

In [ ]:
tab10 = sns.color_palette('tab10')

In [ ]:
plt.figure(figsize = (10, 6))

ax = (dirt_ft[['race_id', 'track_id', 'program_number', 'weight_carried']]
        .drop_duplicates()
         .groupby('track_id')['weight_carried']
         .mean()
         .plot(kind = 'bar', color = tab10[:3]))

ax.bar_label(ax.containers[0])
ax.set_title('DIRT_FT : Stadium and average weight')
plt.xticks(rotation = 0)

#### Approach 2. Number of competitors
- Wouldn't it be slower on average because the more people participate in each game, the more distractions they get?

In [ ]:
ax = (dirt_ft[['race_id', 'track_id', 'program_number']]
        .drop_duplicates()
         .groupby(['track_id', 'race_id'])['program_number']
         .count()
         .groupby('track_id')
         .mean()
          .plot(kind = 'bar', color = tab10[:3])
     )
ax.set_title('DIRT_FT : Average Participants')
ax.bar_label(ax.containers[0])

#### Is a decrease in the number of people and average speed a common phenomenon?
- There is a tendency like that, but I don't think it is enough to make a difference of about 3km/h.
- In addition, **Tends to have a large number of participants in long-distance races**

In [ ]:
temp2 = (stakes[['race_id', 'program_number']].drop_duplicates()
                                                    .groupby('race_id')['program_number']
                                                    .count()
                                                    .to_frame()
                                                    .reset_index()
                                                    .rename(columns = {'program_numbers' : 'members'})
                )
members_races = temp2.groupby('program_number')['race_id'].count().to_frame().reset_index().rename(columns = {'program_number' : 'participants','race_id' : 'races'})

race_members = (stakes[['race_id', 'program_number']].drop_duplicates()
                                                    .groupby('race_id')['program_number']
                                                    .count()
                                                    .to_frame()
                                                    .reset_index()
                                                    .rename(columns = {'program_numbers' : 'members'})
                )
race_speed = (stakes[['race_id', 'velocity_km/h']].groupby(['race_id'])['velocity_km/h']
                                                      .mean()
                                                      .to_frame()
                                                      .reset_index()
                                                      .rename(columns = {'velocity_km/h' : 'avg_speed'})
)

race_members_speed = race_members.merge(race_speed, on = 'race_id')
race_members_speed = race_members_speed.groupby('program_number')['avg_speed'].mean().to_frame().reset_index()


race_dist = (stakes[['race_id', 'race_distance_km']].groupby(['race_id'])['race_distance_km']
                                                      .mean()
                                                      .to_frame()
                                                      .reset_index()
                                                      .rename(columns = {'race_distance_km' : 'avg_distance'})
)
race_members_dist = race_members.merge(race_dist, on = 'race_id')
race_members_dist = race_members_dist.groupby('program_number')['avg_distance'].mean().to_frame().reset_index()


# 시각화
fig, ax = plt.subplots(1, 3, figsize = (16, 9))

sns.barplot(data = members_races,
           x = 'participants',
           y = 'races',
           ax = ax[0])

ax[0].set_title('Overall : # of Participants and Races')
ax[0].bar_label(ax[0].containers[0])


sns.barplot(data = race_members_speed, x = 'program_number', y = 'avg_speed', ax = ax[1])
ax[1].set_title('Overall : # of Participants and Average Speed')
ax[1].set_ylim([65, 70])

sns.barplot(data = race_members_dist, x = 'program_number', y = 'avg_distance', ax = ax[2])
ax[2].set_title('Overall : # of Participants and Average Distance')
ax[2].set_ylim([1.25, 2.1])

In [ ]:
temp2 = (dirt_ft[['race_id', 'program_number']].drop_duplicates()
                                                    .groupby('race_id')['program_number']
                                                    .count()
                                                    .to_frame()
                                                    .reset_index()
                                                    .rename(columns = {'program_numbers' : 'members'})
                )
members_races = temp2.groupby('program_number')['race_id'].count().to_frame().reset_index().rename(columns = {'program_number' : 'participants','race_id' : 'races'})

race_members = (dirt_ft[['race_id', 'program_number']].drop_duplicates()
                                                    .groupby('race_id')['program_number']
                                                    .count()
                                                    .to_frame()
                                                    .reset_index()
                                                    .rename(columns = {'program_numbers' : 'members'})
                )
race_speed = (dirt_ft[['race_id', 'velocity_km/h']].groupby(['race_id'])['velocity_km/h']
                                                      .mean()
                                                      .to_frame()
                                                      .reset_index()
                                                      .rename(columns = {'velocity_km/h' : 'avg_speed'})
)

race_members_speed = race_members.merge(race_speed, on = 'race_id')
race_members_speed = race_members_speed.groupby('program_number')['avg_speed'].mean().to_frame().reset_index()

# # 시각화
# fig, ax = plt.subplots(1, 2, figsize = (16, 9))

# sns.barplot(data = members_races,
#            x = 'participants',
#            y = 'races',
#            ax = ax[0],
#            color = dirt_palette[0])


# ax[0].set_title('DIRT_FT : # of Participants and Races')
# ax[0].bar_label(ax[0].containers[0])


# sns.barplot(data = race_members_speed, x = 'program_number', y = 'avg_speed', color = dirt_palette[1], ax = ax[1])
# ax[1].set_title('DIRT_FT : # of Participants and Average Speed')
# ax[1].set_ylim([65, 70])

In [ ]:
# 주행거리가 영향을 미치니까 주행거리도 살펴보자 : x축은 레이스 당 참가인원 수, y축은 주행 거리 및 평균 속도
temp = dirt_ft[['race_id', 'program_number', 'race_distance_km']].drop_duplicates()
temp['participants'] = temp.groupby(['race_id'])['program_number'].transform('count')
temp = temp.drop('program_number', axis = 1).drop_duplicates()
temp

In [ ]:
# 레이스의 평균 속도도 뽑자
temp2 = dirt_ft[['race_id', 'program_number', 'velocity_km/h']]
temp2['race_avg_velocity'] = temp2.groupby('race_id')['velocity_km/h'].transform('mean')
temp2 = temp2.drop(['program_number', 'velocity_km/h'], axis = 1).drop_duplicates() 
temp2.head()

In [ ]:
temp3 = temp.merge(temp2, on = 'race_id')
temp3.head(1)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (16, 9))

sns.barplot(data = members_races,
           x = 'participants',
           y = 'races',
           ax = ax[0],
           color = dirt_palette[0])

ax[0].set_title('DIRT_FT : # of Participants and # of Races')
# ax[0].bar_label(ax[0].containers[0])

sns.barplot(data = race_members_speed.rename(columns = {'program_number' : 'participants'}), x = 'participants', y = 'avg_speed', color = dirt_palette[1], ax = ax[1])
ax[1].set_title('DIRT_FT : # of Participants and Average Speed')
ax[1].set_ylim([65, 70])

sns.barplot(data = temp3, x = 'participants', y = 'race_distance_km', ci = False, ax = ax[2], color = dirt_palette[2])
ax[2].set_ylim([1.2, 2.2])
ax[2].bar_label(ax[2].containers[0])
ax[2].set_title('DIRT_FT : # of Participants and Average Distance')


- It is a graph about the number of games, average speed, and distance.
- When comparing participants 6 and 8 with the most similar average distance (6m difference), it can be seen that the average speed of 8 is low
- It seems necessary to compare values that have the same distance and have different numbers of participants

In [ ]:
dist_values = temp3['race_distance_km'].value_counts()
dist_mode_idx = dist_values.index[:5]

In [ ]:
fig, ax = plt.subplots(len(dist_mode_idx), 1, figsize = (7, 30))

for i in range(len(dist_mode_idx)):
    t = (temp3[temp3['race_distance_km'] == dist_mode_idx[i]][:]
                                                             .groupby('participants')['race_avg_velocity']
                                                             .mean()
                                                             .to_frame()
                                                            .reset_index()
        )
    sns.barplot(data = t, x = 'participants', y = 'race_avg_velocity', color = 'brown', ax = ax[i])
    ax[i].set_ylim([60, 72])
    ax[i].bar_label(ax[i].containers[0])
    ax[i].set_title(f"DIRT_FT : When Distance Race = {round(dist_mode_idx[i], 2)}km / samples : {dist_values[dist_mode_idx[i]]}")
plt.xticks(rotation = 0)

- It will be difficult to conclude because each race distance has different patterns.

- That's all I've done.
- If you have any advice, please leave a comment. That'll be really helpful for me. Thank you.